- sampling_by_rating에서는 시간의 한계와 모델을 더 발전 시킬 필요성을 확인하기 위해 16만개의 report중 약 20000개를 sample로 지정 \
- sampling 기준은 투자의견을 [strongbuy. buy. neutral, sell]로 분류하고 이 기준을 바탕으로 sampling 진행

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from konlpy.tag import Kkma
from konlpy.tag import Komoran
from konlpy.tag import Hannanum
from konlpy.tag import Okt

In [4]:
from eunjeon import Mecab

In [5]:
import time

In [5]:
okt = Okt()
mecab = Mecab()

In [3]:
df = pd.read_csv('drop_error_txt_df.csv', index_col = 0)
df.index = range(len(df))
df.code = df.code.apply(lambda x :str(x).zfill(6))
df.head()

,작성일,code,제목,목표주가,투자의견,작성자,제공출처,idx
0,2021-11-03,323410,카카오뱅크(323410)포트폴리오 급변 시기에는 순마진율을 봐야,0,Not Rated,김도하,한화투자증권,600357
1,2021-11-03,006400,삼성SDI(006400)EV 향 원형전지가 효자,860000,Buy,이순학,한화투자증권,600354
2,2021-11-03,084650,랩지노믹스(084650)[3Q21 Review 2020 년 연간 실적을 넘어서는 ...,57000,Buy,김형수,한화투자증권,600353
3,2021-11-03,323410,카카오뱅크(323410)펀더멘탈은 변함이 없다,81000,Buy,김현기,하이투자증권,600352
4,2021-11-03,006400,삼성SDI(006400)실적 성장세는 계속된다,1000000,Buy,정원석,하이투자증권,600351


In [4]:
# 투자의견 중 같은 단어지만 대소문자 or ' '가 존재하여 수정

df['투자의견'] = df.투자의견.apply(lambda x : x.lower())
df['투자의견'] = df.투자의견.apply(lambda x : x.replace(' ',''))
df.투자의견.value_counts().index.tolist()

['buy',
 '매수',
 'notrated',
 'hold',
 '-',
 'marketperform',
 '중립',
 'tradingbuy',
 'nr',
 'strongbuy',
 'outperform',
 'neutral',
 '시장평균',
 '적극매수',
 'n/a',
 'reduce',
 'underperform',
 'mktper',
 'sell',
 'suspended',
 '축소',
 '비중축소',
 'na',
 '보유',
 '비중확대',
 '투자의견없음']

In [77]:
df.투자의견.nunique()  # 기존에 42개의 투자의견이 26로 축소

26

In [5]:
# 26종류의 투자의견을 4가지로 분류

rat_4 = ['strongbuy','적극매수','outperform']
rat_3 =['buy','매수','tradingbuy','비중확대']
rat_2 =['hold','중립','marketperform','neutral','시장평균','보유','mktper']
rat_1 = ['reduce','underperform','매도','비중축소','sell','suspended','축소',\
         '-','n/a','na','투자의견없음','notrated','nr']


In [6]:
op_list = []

for i in df.투자의견 :
    if i in rat_4:
        i = 'strongbuy'
        
    elif i in rat_3:
        i = 'buy'
        
    elif i in rat_2:
        i = 'neutral'
        
    else : 
        i = 'sell'
        
    op_list.append(i)
    
df['투자의견'] = op_list

In [7]:
df['투자의견'].value_counts()

buy          124692
sell          14253
neutral       11543
strongbuy      1261
Name: 투자의견, dtype: int64

### 160,000 ==> 20,000 sampling

In [8]:
# 가장 데이터가 적은 strongbuy는 모두 포함시키고 나머지는 적절한 비율로 sampling

df_temp = df.copy()
df_buy = df_temp[df_temp.투자의견 == 'buy'].sample(n=10000)
df_sell = df_temp[df_temp.투자의견 == 'sell'].sample(n=5000)
df_neutral = df_temp[df_temp.투자의견 == 'neutral'].sample(n=5000)
df_strongbuy = df_temp[df_temp.투자의견 == 'strongbuy']


df_list = [df_buy,df_sell,df_neutral,df_strongbuy]
for n,i in enumerate(df_list) :
    if n != 0 :
        df_buy = pd.concat([df_buy,i])
df_sampled_buy = df_buy.copy()

In [96]:
df_sampled_buy.투자의견.value_counts() 

buy          10000
sell          5000
neutral       5000
strongbuy     1261
Name: 투자의견, dtype: int64

In [9]:
df_sampled_buy

,작성일,code,제목,목표주가,투자의견,작성자,제공출처,idx
128913,2013-07-02,000100,유한양행 (000100),250000,buy,"신정현,이한엽",삼성증권,306539
90535,2016-03-25,100120,뷰웍스(100120),57000,buy,"이지용,배기달",신한금융투자,375389
104685,2015-04-09,004020,현대제철(004020),87000,buy,남광훈,교보증권,347231
96030,2015-10-30,009150,삼성전기(009150),80000,buy,노근창,HMC투자증권,364394
96697,2015-10-19,006400,삼성SDI(006400),130000,buy,이학무,미래에셋증권,363236
...,...,...,...,...,...,...,...,...
151362,2011-11-14,000830,삼성물산 (000830),110000,strongbuy,"김열매,박판근",현대증권,272705
151440,2011-11-11,097230,한진중공업(097230),40000,strongbuy,서정덕,메리츠증권,272587
151453,2011-11-10,000210,대림산업 (000210),166000,strongbuy,"이왕상,이창희",우리증권,272570
151630,2011-11-07,010620,현대미포조선 (010620),143000,strongbuy,정동익,한화증권,272294


In [97]:
df_sampled_buy.nunique()

작성일      2446
code     1643
제목      11721
목표주가      870
투자의견        4
작성자      1910
제공출처       39
idx     21261
dtype: int64

In [114]:
df_sampled_buy.to_csv('sampled_df.csv')

In [98]:
df1 = df_sampled_buy[['작성일','code','목표주가','idx']]
df1

,작성일,code,목표주가,idx
50639,2018-04-18,088350,11000,473108
87611,2016-05-23,092070,17000,380646
113571,2014-07-29,000270,77000,329427
32806,2019-04-03,035720,135000,519412
81924,2016-09-12,020560,5300,391416
...,...,...,...,...
151362,2011-11-14,000830,110000,272705
151440,2011-11-11,097230,40000,272587
151453,2011-11-10,000210,166000,272570
151630,2011-11-07,010620,143000,272294


In [113]:
import os

In [100]:
# txt_path = './report_text/txt/'
# path = os.listdir(txt_path)
# Series_txt_path = pd.Series(path)
# Series_txt_path

0           20111104_BS금융_272144.txt
1           20111104_BS금융_272186.txt
2           20111104_BS금융_272198.txt
3         20111104_BS금융지주_272152.txt
4         20111104_BS금융지주_272158.txt
                     ...            
152000     20211103_카카오뱅크_600352.txt
152001     20211103_카카오뱅크_600357.txt
152002    20211103_한국금융지주_600311.txt
152003    20211103_한국금융지주_600319.txt
152004     20211103_효성중공업_600341.txt
Length: 152005, dtype: object

###  Tokenizing(Okt) Sampled DataFrame 

In [112]:
# 각 report를 tokenizing하여 list에 담고 column으로 포함시킴
# tokenizing시 어떤 형태소분석기를 사용할지 비교

%%time
txt_path = './report_text/txt/'
path = os.listdir(txt_path)
Series_txt_path = pd.Series(path)

def han_text_clean(text):
    text = re.sub("[^가-힣ㄱ-ㅎa-zA-Z ]", "", text)
   
    return text

def idx_to_open_txt(path, idx):
    txt_file = Series_txt_path[Series_txt_path.str.contains(idx)].values[0]
    with open(path+txt_file, 'r', encoding='utf-8') as f:
        read=f.readlines()
    return read



import re
with open('stopwords-ko.txt', 'r', encoding = 'utf-8') as f:
    sw1 = f.read()
    sw1 = re.sub('\\n', ' ', sw1)
    sw1 = sw1.split()

stop_report = '리서치센터  Compliance Notice 본 조사분석자료는 제3자에게 사전 제공된 사실이 없습니다. 당사는 자료작성일 현재 본 조사분석자료에 언급된 종목의 지분을 1% 이상 보유하고 있지 않습니다. \
본 자료를 작성한 애널리스트는 자료작성일 현재 해당 종목과 재산적 이해관계가 없습니다. 본 자료에 게재된 내용은 본인의 의견을 정확하게 반영하고 있으며, 외부의 부당한 압력이나 간섭 없이 신의 성실하게 작성되었음을 확인합니다. \
본 자료는 투자자들의 투자판단에 참고가 되는 정보제공을 목적으로 배포되는 자료입니다. 본 자료에 수록된 내용은 당사 리서치센터의 추정치로서 오차가 발생할 수 있으며 정확성이나 완벽성은 보장하지 않습니다. \
본 자료를 이용하시는 분은 본 자료와 관련한 투자의 최종 결정은 자신의 판단으로 하시기 바랍니다. 따라서 어떠한 경우에도 본 자료는 투자 결과와 관련한 법적 책임소재의 증빙자료로 사용될 수 없습니다. \
본 조사분석자료는 당사 고객에 한하여 배포되는 자료로 당사의 허락 없이 복사, 대여, 배포 될 수 없습니다.\
투자등급 이 자료에 게재된 내용들은 본인의 의견을 정확하게 반영하고 있으며, 외부의 부당한 압력이나 간섭없이 작성되었음을 확인합니다.(작성자 : 하준두) 자료 제공일 현재\
당사는 상기 회사가 발행한 주식을 1% 이상 보유하고 있지 않습니다. 자료 제공일 현재 당사는 지난 1년간 상기 회사의 최초 증권시장 상장시 대표 주관사로 참여한 적이 없습니다. \
자료제공일 현재 조사분석 담당자는 상기회사가 발행한 주식 및 주식관련사채에 대하여 규정상 고지하여야 할 재산적 이해관계가 없으며, 추천의견을 제시함에 있어 어떠한 금전적 보상과도 연계되어 있지 않습니다. \
당자료는 상기 회사 및 상기회사의 유가증권에 대한 조사분석담당자의 의견을 정확히 반영하고있으나 이는 자료제공일 현재 시점에서의 의견 및 추정치로서 실적치와 오차가 발생할 수 있으며, \
투자를 유도할 목적이 아니라 투자자의 투자판단에 참고가 되는 정보제공을 목적으로 하고 있습니다. 따라서 종목의 선택이나 투자의 최종결정은 투자자 자신의 판단으로 하시기 바랍니다. \
Analyst, 증권,한화투자증권, 하이투자증권, 대신증권, SK증권, IBK투자증권, 키움증권, 이베스트증권,유안타증권, 메리츠증권, 유진투자증권, 카카오페이증권, 한양증권, 하나금융투자, 교보증권, \
현대차투자증권, 메리츠종금증권, 삼성증권, KTB투자증권, 신한금융투자, 한국투자증권,미래에셋대우, KB증권, 미래에셋증권, 하나대투증권, HMC투자증권, NH농협증권, 현대증권,\
동양증권, KDB대우증권, 이트레이드증권, 우리투자증권, 하이증권, 우리증권, 신한금융,대우증권, NH증권, 유진증권, 한투증권, 한화증권, 한화투자증권, 작성자, 한화리서치,기업, 코멘트'

stop_report = re.sub("[^가-힣ㄱ-ㅎa-zA-Z ]", "", stop_report)
  
sw2 = okt.nouns(han_text_clean(stop_report))   # stopwords : okt

stopwords = sw1 + sw2





# 우선 100개만 test
idx_list = df1.idx.values.tolist()[:100]
okt_token_list = []

for n,i in enumerate(idx_list) :
    report_temp = idx_to_open_txt(txt_path,str(i))
    clean_report = han_text_clean(report_temp[0])
    tokens = okt.nouns(clean_report)
    
    clean_tokens = [w for w in tokens if (not w in stopwords) & (len(w) > 1)]
    okt_token_list.append(clean_tokens)
    
    print('index : ',n, ',', 'idx : ', i)
    print(clean_tokens)
    print()
    print('=========================================================================================================================')

index :  0 , idx :  473108
['건설', '유틸리티', '생명', '이슈', '브리프', '보험', '조원', '신종', '자본', '성공', '컨센서스', '대비', '상회', '부합', '평균', '거래', '대금', '억원', '외국인', '분율', '절대', '수익률', '상대', '수익률', '배당', '수익률', '재무', '데이터', '십억원', '매출', '영업', '이익', '순이익', '생명', '생명', '목표', '주가', '주가', '시가총액', '규모', '해외', '신종', '자본', '성공', '동사', '지난해', '국내', '억원', '전일', '달러', '규모', '해외', '신종', '자본', '언론보도', '기관', '입찰', '경쟁률', '기록', '신종', '자본', '상세', '만기', '연장', '가능', '조기상', '환권', '포함', '일로', '이후', '개월', '마다', '청구', '가능', '이자율', '고정', '이자율', '산정', '근거', '전일', '미국', '국채', '금리', '가산', '금리', '합산', '이후', '년물', '국고', '금리', '가산', '금리', '스텝업', '금리', '추가', '지급', '이자', '누적', '납입', '동사', '분기', '비율', '상승', '예상', '미국', '금리', '상승', '추세', '지속', '전망', '때문', '도입', '대비', '선제', '대응', '긍정', '목표', '주가', '업종', '최선', '호주', '유지', '이슈', '브리프', '생명', '생명', '목표', '주가', '추이', '변동', '일시', '목표', '가격', '괴리율', '일시', '목표', '가격', '괴리율', '최고', '대비', '최저', '대비', '평균', '대비', '최고', '대비', '최저', '대비', '평균', '대비', '변경', '박혜진']

index :  1 , idx :  380646
['반도체'

index :  4 , idx :  391416
['아시아나', '항공', '아시아나', '항공', '운송', '엔터테인먼트', '정유석', '유상증자', '억원', '하향', '목표', '주가', '하향', '기적', '상황', '유상증자', '심리', '영향', '영업', '이익', '억원', '전망', '액면가', '최고', '보통주', '최저', '보통주', '자본금', '억원', '시가총액', '억원', '보통주', '만주', '우선주', '만주', '평균', '거래', '량일', '만주', '평균', '거래', '금일', '억원', '외국', '인지', '보통주', '주주', '금호', '산업', '금호', '석유화학', '주가수익률', '개월', '개월', '개월', '절대', '주가', '상대', '주가', '하향', '목표', '주가', '하향', '예정', '기준', '동사', '증가', '하락', '예상', '식수', '증가', '고려', '목표', '주가', '기존', '대비', '제시', '하향', '조정', '기적', '상황', '유상증자', '심리', '영향', '지난주', '금요일', '아시아나', '항공', '억원', '규모', '유상증자', '실시', '공시', '증자', '방식', '배정', '증자', '보통주', '예정', '신주', '조합원', '배정', '운영', '자금', '기존', '식수', '대비', '가량', '수가', '증가', '때문', '주가', '하락', '상황', '동사', '상반기', '누적', '기준', '영업', '이익', '억원', '기록', '기조', '유지', '항공', '운송업', '개선', '이후', '주가', '상승', '경험', '유상증자', '그룹', '지원', '이슈', '심리', '영향', '전망', '자금', '달이', '동사', '부채', '비율', '재무', '안정', '위해', '입금', '상환', '산업', '은행', '금호', '타이어', '매각', '이슈', '상황', 

index :  8 , idx :  572444
['프레', '테이', '서영수', '유근', '가계부채', '부채', '위기', '가능성', '점검', '가계', '부채', '진단', '은행', '영향', '은행', '산업', '소외', '이유', '증권사', '부문', '위험', '정부', '정책', '전망', '증권사', '위험관리', '능력', '코로나', '위기', '시사', '이제', '투자가', '대세', '기준', '이익', '안정', '유망', '주가', '외면', '이유', '개선', '위험', '수익', '기반', '이익', '비중', '증가', '수익', '모델', '중개', '전환', '위탁', '중개', '사업', '사양', '산업', '인식', '서비스', '확대', '가격', '경쟁', '치중', '부동산', '사모펀드', '과도', '리스크', '시스템', '전환', '증권사', '취약', '위험관리', '능력', '코로', '나발', '유동성', '위기', '원인', '코로나', '사태', '급락', '외화', '유동성', '위기', '연결', '부문', '확대', '증권사', '과도', '리스크', '금융위기', '근원', '요인', '인식', '당국', '파생', '결합', '부동산', '사모펀드', '그동안', '육성', '대상', '부문', '규제', '대상', '전환', '사모펀드', '규제', '파생', '결합', '규제', '부동산', '규제', '강화', '부문', '수익', '점차', '둔화', '전망', '이제', '투자가', '대세', '코로나', '사태', '이후', '주가', '급락', '정부', '적극', '유동성', '확대', '정책', '개인', '급증', '위탁', '수수료', '급성', '국내', '지배구조', '불신', '배당', '성향', '불만', '세대', '중심', '해외', '선호', '급증', '수수료', '해외', '중심', '세대', '신규', '급증', '증권사', '수익', '

index :  10 , idx :  319199
['하이닉스', '도움', '동의', '신뢰', '데이터', '근거', '완전', '증거물', '인용', '거나', '의사', '아래', '리기', '하이닉스', '역시', '메모리', '매수', '유지', '메모리', '호황', '지속', '개선', '주주', '이익', '환원', '전망', '분기', '영업', '이익', '억원', '예상', '목표', '주가', '주가', '최고', '최저', '시가총액', '십억원', '자본금', '십억원', '매수', '유지', '목표', '주가', '원유', '최근', '주가', '약세', '매수', '기회', '첫째', '스마트폰', '성장', '둔화', '업종', '성장', '우려', '제기', '상황', '메모리', '구조조정', '효과', '수요', '둔화', '극복', '신규', '수요', '기반', '확보', '성장', '상대', '매력', '둘째', '메모리', '호황', '지속', '플래시', '경쟁력', '강화', '때문', '주가', '올해', '기준', '부담', '제한', '메모리', '호황', '지속', '메모리', '업종', '호황', '장기', '예상', '분기', '하이닉스', '우시', '공장', '가동', '물량', '증가', '일시', '공급', '과잉', '연간', '전체', '공급', '부족', '전망', '호황', '불구', '업체', '점유', '확대', '설비', '수익', '유지', '초점', '급량', '조절', '능력', '때문', '분기', '영업', '이익', '억원', '매출', '조원', '영업', '이익', '억원', '예상', '우시', '공장', '가동', '정상화', '하량', '증가', '고정', '비용', '부담', '때문', '공급', '정상화', '내외', '가격', '하락', '예상', '하량', '증가', '복구', '비용', '부담', '개선', '전망', '식수', '만주', '액면가', '

index :  13 , idx :  485563
['우려', '양호', '우려', '우려', '우려', '우려', '분기', '기분', '분기', '분기', '지수', '하락', '하락', '장대비', '낙폭', '확대', '종가', '기준', '분기', '대비', '지수', '하락', '구간', '별로', '횡보장', '불구', '남북정상회담', '거치', '경협', '대감', '평균', '거래', '금은', '분기', '조원', '증가', '북미', '정상회담', '기점', '경협주', '대감', '하락', '선위', '이슈', '무역', '분쟁', '부각', '글로벌', '증시', '변동성', '확대', '그간', '대북', '경협', '이슈', '국내', '경기', '우려', '부각', '달러', '환율', '급상승', '약세', '거래', '금은', '조원', '급감', '중순', '업종', '비중', '확대', '비중', '확대', '비중', '확대', '비중', '확대', '유지유지', '유지유지', '지금', '업종', '비중', '확대', '유지', '분기', '양호', '유니버스', '개사', '합산', '분기', '순이익', '분기', '대비', '감소', '분기', '법인세', '요인', '제외', '부진', '정도', '만회', '분기', '거래', '대금', '신용', '조기상', '환액', '기초', '지표', '분기', '대비', '개선', '문제', '분기', '주가', '과도', '환율', '안정', '거래', '금은', '조원', '조조원', '반등', '추정', '하반기', '용공', '개선', '지속', '조기', '상환', '이익', '감소', '일부', '상쇄하', '거래', '이익', '증가', '고려', '지금', '브로커', '리지', '브로커', '리지', '브로커', '리지', '브로커', '리지', '수익', '수익', '수익', '수익', '분기', '대비', '분기', '대비', '분기', '대비', '

...

index :  91 , idx :  371020
['이노', '현상', '해결', '체질', '개선', '영업', '이익', '억원', '부진', '이노텍', '매출', '억원', '영업', '이익', '억원', '기록', '매출', '억원', '영업', '이익', '억원', '대치', '매출', '억원', '영업', '이익', '억원', '미국', '스마트폰', '업체', '납품', '기대', '광학', '솔루션', '사업', '카메라', '모듈', '매출', '억원', '억원', '미달', '업황', '악화', '매출', '억원', '외형', '감소', '광학', '솔루션', '사업', '수익', '악화', '장기', '수당', '억원', '일시', '점도', '영업', '이익', '하락', '원인', '사업', '적자', '감소', '자동차', '부품', '사업', '성장', '기대', '자동차', '부품', '사업', '성장', '긍정', '차량', '통신', '모듈', '북미', '공급', '지속', '증가', '차량', '모터', '신규', '차종', '확대', '적용', '년말', '누적', '수주', '잔고', '조원', '조원', '조원', '증가', '수치', '자동차', '부품', '사업', '성장', '미래', '성장', '동력', '매력', '제품', '변화', '다변', '때문', '스마트폰', '자동차', '부품', '파악', '그룹', '자동차', '부품', '산업', '육성', '성장', '가능성', '더욱', '사업', '대폭', '개선', '산업', '조명', '사업', '경쟁', '심화', '가능성', '크기', '때문', '감가상각비', '감소', '효과', '비용', '구조', '효율', '통해', '적자', '감소', '전망', '사업', '업적', '억원', '원로', '추정', '감가상각비', '억원', '억원', '수준', '감소', '효과', '때문', '매수', '유지', '개월', '목표', '주가', '조

index :  93 , idx :  560204
['프레', '테이', '글로벌', '반딧불', '시리즈', '반도체', '디스플레이', '전망', '반도체', '디스플레이', '반도체', '디스플레이', '국내', '업체', '미국', '전략', '디스플레이', '산업', '전망', '디스플레이', '전망', '디스플레이', '중소', '패널', '전망', '디스플레이', '대형', '패널', '전망', '중국', '디스플레이', '산업', '동향', '디스플레이', '산업', '전략', '반도체', '산업', '전망', '산업', '전망', '산업', '전망', '반도체', '업종', '전략', '국내외', '국내', '하이닉스', '머티리얼즈', '한솔', '케미칼', '리노', '공업', '에스에프', '에이', '시스템', '덕산', '룩스', '반도체', '해외', '홀딩', '어플', '라이드', '머티리얼즈', '국내', '업체', '미국', '전략', '반도체', '박유악', '디스플레이', '김소원', '원주', '홍록기', '김진영', '반도체', '상반기', '장비', '집중', '대규모', '집행', '하반기', '상반기', '하반기', '투자가', '크게', '축소', '연말', '중심', '집중', '예상', '하이닉스', '증설', '공급', '증가', '수요', '개선', '업황', '회복', '가시', '주가', '방향성', '전망', '중소형주', '증설', '장비', '물량', '증가', '부품', '선호', '장비', '연말', '가시', '확보', '접근', '권고', '디스플레이', '디스플레이', '조원', '조원', '상승', '사이클', '재개', '전망', '디스플레이', '중소', '침투', '증가', '더블', '확대', '대응', '위해', '규모', '장비', '착수', '예상', '한편', '대형', '사업', '셧다운', '예상', '진행', '전망', '착수', '예상', '중국', '규모', '장

index :  95 , idx :  375037
['카드', '양호', '전망', '지배구조', '개편', '수혜', '최정욱', '박찬주', '매수', '유지', '목표', '주가', '유지', '주가', '카드', '업종', '매수', '목표', '주가', '유지', '목표', '주가', '년말', '추정', '대비', '목표', '적용', '산출', '카드', '분기', '추정', '억원', '컨센서스', '상회', '전망', '예상', '양호', '추정', '배경', '신용판매', '가맹', '점수', '수료', '인하', '불구', '신판', '취급', '고가', '대비', '가량', '증가', '르노', '비롯', '배당', '확대', '영업', '수익', '대비', '개선', '추정', '적극', '판관', '관리', '가맹', '점수', '수료', '인하', '수익', '감소', '효과', '상쇄될', '때문', '가맹', '점수', '수료', '인하', '대비', '감소', '가정', '분기', '우려', '전망', '예상', '취급', '증가', '율과', '판관', '감축', '노력', '감안', '이익', '추정', '기존', '억원', '억원', '상향', '조정', '이익', '상향', '불구', '개선', '효과', '목표', '주가', '유지', '이익', '전망', '상향', '예상', '배당금', '상향', '배당', '성향', '가정', '주가', '기준', '배당', '수익률', '수준', '우려', '양호', '배당', '성향', '상회', '배당', '매력', '장기', '매력', '최근', '주가', '급등', '분할', '합병', '가능성', '그룹', '열사', '지배구조', '개편', '과정', '수혜', '대감', '크게', '때문', '보임', '기대', '논리', '필요', '카드', '분할', '합병', '논리', '제약', '매각', '기정', '문제', '최근', '생명', '인적', '분할', '통한

index :  97 , idx :  414429
['표지', '클라우드', '클라우드', '클라우드', '클라우드', '가시대', '가시대', '가시대', '린다', '린다', '린다', '린다', '중소형주', '상황', '이슈', '리포트', '클라우드', '시대', '린다', '팀팀팀팀', '스몰', '제목', '월보', '형식', '유망', '중소형주', '안정', '현금', '흐름', '메리트', '중심', '중소형주', '유망', '선별', '중장', '기적', '경쟁력', '환경', '부합', '성장', '진행', '중심', '중소형주', '유망', '선별', '구성', '선별', '기준', '적용', '성적', '양하', '지속', '업데이트', '통해', '중장', '기적', '제시', '위함', '중소형주', '환경', '의사결정', '도움', '드릴', '스몰', '캡팀', '노력', '중소형주', '상황', '중소형주', '탐방', '노트', '에스', '지투알', '웅진', '씽크빅', '코오롱', '플라스틱', '고영', '월월월월', '선별', '방식', '적용', '중소형주', '구성', '유망', '제시', '신규', '편입', '제외', '업데이트', '변화', '신규', '정리', '포인트', '요약', '코드', '코드', '코드', '코드', '포인트', '포인트', '포인트', '포인트', '정홍', '리노', '공업', '소켓', '추세', '성장', '글로벌', '경쟁력', '기반', '성장', '유지', '단자', '가치', '체크', '포인트', '상승', '수출', '자동차', '커넥터', '비용', '향후', '매출', '선행', '지표', '대교', '한국사', '수업', '신규', '출시', '모멘텀', '영업', '자산', '시가총액', '근접', '메리트', '평가', '신용정보', '성장', '개인', '부문', '회복', '대감', '금화', '스시', '안정', '재무', '구조', '민간', '

index :  99 , idx :  385088
['분기', '무선', '가입자', '인터넷', '가입자', '가입자', '매출', '영업', '이익', '변환', '비교', '무선', '가입자', '인터넷', '가입자', '가입자', '매출', '영업', '이익', '직전', '최근', '무선', '가입자', '인터넷', '가입자', '가입자', '천명', '단위', '매출', '영업', '이익', '연결', '기준', '십억원', '단위', '통신', '서비스', '여러', '이벤트', '안정', '통신', '서비스', '중립', '유지', '매수', '유지', '목표', '주가', '매수', '유지', '목표', '주가', '주가', '추이', '주가', '추이', '유플러스', '이벤트', '동요', '매달', '스마트폰', '모델', '출시', '가입자', '생각', '안정', '스마트폰', '교체', '주기', '가입자', '신규', '가입', '매력', '디바이스', '분기', '마케팅', '비용', '주력', '모델', '업그레이드', '반응', '모델', '감안', '정도', '안정', '표현', '통신', '합산', '매출', '대비', '증가', '영업', '이익', '감소', '추측', '지난해', '분기', '분기', '마케팅', '여파', '마케팅', '매우', '안정', '분기', '대비', '증가', '추정', '하반기', '이벤트', '안정', '유지', '상반기', '예고', '이벤트', '텔레콤', '헬로비전', '인수합병', '미해결', '나머지', '소멸', '하반기', '예고', '이벤트', '스마트폰', '모델', '출시', '확정', '출시', '상반기', '영향력', '예상', '통신', '안정', '규제', '당국', '의지', '감안', '가입자', '안정', '유지', '통신사', '무난', '흐름', '전망', '뉴스', '업종', '규제', '업종', '규제', '이슈', '주가', '경향', '하반기',

In [ ]:
# 예상 소요시간 2시간 

In [ ]:
# 100개 소요시간 - okt - 38.1s
# 총 예상 소요시간 2시간 
# 우리 조는 okt

In [ ]:
# 100개 소요시간 - mecab - 10s
# 총 예상 소요시간 33분


In [ ]:
# 100개 소요시간 - han - 1min 18s
# 총 예상 소요시간 4 시간 20분 


In [ ]:
# 100개 소요시간 - kom - 2min 10s
# 총 예상 소요시간 4 시간 20분 
